## Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

# check if workding_dir is in local variables
if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)

os.chdir(workding_dir)
sys.path.append(workding_dir)
# print("workding dir:", workding_dir)

## Generate Paper Results

In [9]:
# Step 1: Process raw evaluation results
!python src/analysis/eval_laj_run.py --folder results --all-runs

LLM-as-a-Judge Benchmark CSV Generation with Retry Tracking

🎯 Features:
  - Tracks attempt_number for each evaluation
  - Tracks total_attempts per ticket
  - Supports reliability metrics

🔍 Found 5 run folder(s): ['r1', 'r2', 'r3', 'r4', 'r5']


Processing r1...

  📋 Model Summary:
  Model                                    Attempts   Failed   Success   
  ----------------------------------------------------------------------
  ✓ gpt-5-nanohigh                           2          0        100.0%    
  ✓ gpt-4.1-nano                             2          0        100.0%    
  ✓ gpt-4.1                                  2          0        100.0%    
  ✓ gpt-5-minihigh                           2          0        100.0%    
  ✓ gpt-4o                                   2          0        100.0%    
  ✓ gpt-4.1-mini                             2          0        100.0%    
  ✓ gpt-5-minimedium                         2          0        100.0%    
  ✓ gpt-5-nanolow                   

In [10]:
# Step 2: Calculate metrics (MAAE, APS, ECR@1, costs)
!python src/analysis/cost_benefit_analysis.py

LLM-as-a-Judge Cost-Benefit Analysis V3 - LAJ Paper Metrics
🔍 Found 5 run folder(s): ['r1', 'r2', 'r3', 'r4', 'r5']
  ✓ r1: Found 20 model(s)
  ✓ r2: Found 20 model(s)
  ✓ r3: Found 20 model(s)
  ✓ r4: Found 20 model(s)
  ✓ r5: Found 20 model(s)

✓ Ground truth loaded from dataset/jira_coverage_ground_truth.csv

📊 Analyzing 5 run(s)...

Processing r1...
  ✓ GPT-OSS 20B (high): MAE=19.81, ECR@1=90.0%, Cost=$0.49/1K
  ✓ GPT-5 Mini (low): MAE=7.53, ECR@1=95.0%, Cost=$3.70/1K
  ✓ GPT-4o Mini: MAE=5.94, ECR@1=95.0%, Cost=$0.97/1K
  ✓ GPT-OSS 20B (medium): MAE=18.85, ECR@1=93.0%, Cost=$0.47/1K
  ✓ GPT-5 Nano (low): MAE=9.40, ECR@1=90.0%, Cost=$0.69/1K
  ✓ GPT-5 (medium): MAE=7.05, ECR@1=100.0%, Cost=$46.58/1K
  ✓ GPT-4.1 Nano: MAE=8.60, ECR@1=100.0%, Cost=$0.76/1K
  ✓ GPT-4o: MAE=8.60, ECR@1=100.0%, Cost=$16.79/1K
  ✓ GPT-4.1 Mini: MAE=7.05, ECR@1=100.0%, Cost=$3.07/1K
  ✓ GPT-OSS 120B (low): MAE=13.69, ECR@1=99.0%, Cost=$0.73/1K
  ✓ GPT-5 Mini (high): MAE=7.45, ECR@1=99.0%, Cost=$15.21/1K
 

In [13]:
# Step 3: Generate LaTeX table for paper
!python src/analysis/generate_latex_table.py -o results/combined_results_table.tex

In [15]:
# Print plain text table
import pandas as pd
from tabulate import tabulate

# Read the metrics summary
df = pd.read_csv('results/laj_metrics_summary.csv')

# Select key columns for display
display_cols = ['model', 'maae_mean', 'maae_std', 'aps_mean', 'ecr1_mean', 
                'attempts_mean', 'cost_nominal_mean', 'cost_adjusted_mean']

# Rename columns for better readability
df_display = df[display_cols].copy()
df_display.columns = ['Model', 'MAAE', 'MAAE_std', 'APS', 'ECR@1%', 
                       'Attempts', 'Cost_Nom', 'Cost_Adj']

# Round numeric columns
df_display['MAAE'] = df_display['MAAE'].round(2)
df_display['MAAE_std'] = df_display['MAAE_std'].round(2)
df_display['APS'] = df_display['APS'].round(2)
df_display['ECR@1%'] = df_display['ECR@1%'].round(1)
df_display['Attempts'] = df_display['Attempts'].round(2)
df_display['Cost_Nom'] = df_display['Cost_Nom'].round(2)
df_display['Cost_Adj'] = df_display['Cost_Adj'].round(2)

print("\n" + "="*105)
print("LAJ-Gherkin Metrics Summary")
print("="*105)
print(tabulate(df_display, headers='keys', tablefmt='grid', showindex=False))
print("="*105)


LAJ-Gherkin Metrics Summary
+-----------------------+--------+------------+-------+----------+------------+------------+------------+
| Model                 |   MAAE |   MAAE_std |   APS |   ECR@1% |   Attempts |   Cost_Nom |   Cost_Adj |
+=======================+========+============+=======+==========+============+============+============+
| GPT-4.1               |   8.14 |       0.06 | 91.86 |    100   |       1    |      15.23 |      15.23 |
+-----------------------+--------+------------+-------+----------+------------+------------+------------+
| GPT-4.1 Mini          |   6.92 |       0.1  | 93.08 |     99.8 |       1    |       3.07 |       3.08 |
+-----------------------+--------+------------+-------+----------+------------+------------+------------+
| GPT-4.1 Nano          |   8.61 |       0.14 | 91.39 |     99.8 |       1    |       0.76 |       0.76 |
+-----------------------+--------+------------+-------+----------+------------+------------+------------+
| GPT-4o         